In [28]:
import os
import copy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  

X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.98, random_state=2)
XR, yR = make_regression(n_samples=555, n_features=1, noise=10, random_state=5)
X_train2, X_test2, y_train2, y_test2 = train_test_split(XR, yR, test_size=0.20, random_state=2)

def normalize_dataset(X):
    return tf.keras.utils.normalize(X)



In [75]:
class Layer(tf.Module):
    def __init__(self, activation=tf.identity):
        self.activation = activation
        self.build = False
        self.normalized = False

    @tf.function(reduce_retracing=True)
    def xavier_init(self, shape):
        in_dim, out_dim = shape
        xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
        weight_vals = tf.cast(tf.random.uniform(shape=(in_dim, out_dim), 
                                        minval=-xavier_lim, maxval=xavier_lim, seed=22, dtype=tf.float32), dtype=tf.double)
        return weight_vals

    @tf.function
    def __call__(self, X):
        if not self.build:
            n_samples, n_features = X.shape
            self.weights = tf.Variable(self.xavier_init(shape=(n_features, n_samples)), name="Weights", dtype=tf.double, trainable=True, )
            self.bias = tf.Variable(tf.zeros(shape=1, dtype=tf.double), name="Bias", dtype=tf.double, trainable=True )
            self.build = True

        z = tf.add(tf.matmul(X, self.weights), self.bias)

        print(X.shape,"*",self.weights.shape,"+",self.bias.shape, "=", z.shape)
        return self.activation(z)

    

class MLPLinearRegressor(tf.Module):
    def __init__(self, layers, X, X_test, y, y_test, lr=0.001, epochs=10):
        self.layers = layers
        self.lr = lr
        self.epochs = epochs
        self.loaded = False
        self.X = X 
        self.XN = normalize_dataset(X)
        self.XN_test = normalize_dataset(X_test)
        self.y = y
        self.y_test = y_test
        self.X_test = X_test
        self.learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=1000000, decay_rate=0.96, staircase=True)
        self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)
        self.train_mse = tf.keras.metrics.MeanSquaredError()
        self.train_accuracy = tf.keras.metrics.MeanSquaredLogarithmicError()
        self.train_precision = tf.keras.metrics.Precision()
        self.test_mse = tf.keras.metrics.MeanSquaredError()
        self.test_accuracy = tf.keras.metrics.MeanSquaredLogarithmicError()
        self.test_precision = tf.keras.metrics.Precision()
        self.l1 = tf.keras.regularizers.l1()
        self.l2 = tf.keras.regularizers.l1()
        self.regularizer = tf.keras.layers.ActivityRegularization()
        self.loss_history = [e for e in range(epochs)]
        self.regloss_history = [e for e in range(epochs)]

    @tf.function(reduce_retracing=True)
    def train_accuracy_rrsse(self, predicted):
        return tf.divide(tf.sqrt(
            tf.divide(
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(predicted, self.y))))),
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(tf.reduce_mean(self.y), predicted)))))
            )), predicted.shape[0]
        )

    @tf.function(reduce_retracing=True)
    def loss(self, predicted):
        return tf.add(tf.losses.MSE(self.y, predicted), self.lasso())
    
    @tf.function
    def lasso(self):
        lassos = [w for w in range(len(self.layers))]
        for w in range(len(self.layers)):
            lassos[w] = tf.reduce_sum(tf.norm(self.layers[w].weights))

        return tf.reduce_mean(lassos)

    @tf.function(reduce_retracing=True)
    def calc_metrics(self):
        self.train_accuracy.update_state(self.y, self.predicted)
        self.train_precision.update_state(self.y, self.predicted)
        self.train_mse.update_state(self.y, self.predicted)
        self.test_accuracy.update_state(self.y_test, self.predicted_test)
        self.test_precision.update_state(self.y_test, self.predicted_test)
        self.test_mse.update_state(self.y_test, self.predicted_test)

    @tf.function(reduce_retracing=True)
    def train_step(self, X):
        for layer in self.layers:
            X = layer(X)
        return X

    def train(self):
        self.weights = model.train_step(self.XN)
        self.vars = [self.layers[0].weights, self.layers[1].weights, self.layers[2].weights,self.layers[3].weights,self.layers[4].weights, self.layers[0].bias, self.layers[1].bias, self.layers[2].bias,self.layers[3].bias,self.layers[4].bias]
        self.weights_list = [self.layers[0].weights, self.layers[1].weights, self.layers[2].weights,self.layers[3].weights,self.layers[4].weights,]

        for e in range(self.epochs):
            with tf.GradientTape(watch_accessed_variables=True, persistent=True) as tape:
                self.weights = self.train_step(self.XN)
                self.bias = self.layers[len(self.layers)-1].bias
                self.flatten_weights = tf.reduce_mean(self.weights, axis=0)
                self.predicted = self._predict_train(tf.transpose(self.X), self.flatten_weights)
                
                loss = tf.nn.scale_regularization_loss(self.loss(self.predicted)) 

                self.compressed_weights = tf.reduce_mean(tf.add(tf.multiply(tf.transpose(self.XN), tf.reduce_mean(self.weights, axis=0)), self.bias), axis=1)
                
                self.loss_history[e] = loss.numpy()
                self.regloss_history[e] = loss.numpy()
                

            grads = tape.gradient(loss, self.vars)
            self.optimizer.apply_gradients(zip(grads, self.vars)) 

    def _predict_train(self, X, weights):
        compressed_weights = tf.reduce_mean(tf.subtract(tf.multiply(X, weights), self.bias), axis=1)
        return tf.reduce_mean(tf.add(tf.multiply(tf.transpose(X), compressed_weights), self.bias), axis=1)

    def predict_test(self):
        print(self.compressed_weights)
        self.predicted_test = tf.reduce_mean(tf.add(tf.multiply(self.X_test, self.compressed_weights), self.bias), axis=1)

    def predict(self, x):
        return tf.reduce_mean(tf.add(tf.multiply(x, self.compressed_weights), self.bias))

model = MLPLinearRegressor([
    Layer(),
    Layer(),
    Layer(activation=tf.nn.relu),
    Layer(activation=tf.sigmoid),
    Layer()
], X_train2, X_test2, tf.squeeze(y_train2), tf.squeeze(y_test2), lr=0.01, epochs=60)


model.train()
model.predict_test()
print("Original: ", y_test2[0], "...Predicted: ", model.predict(X_test2[0]).numpy(), "...Tested:", model.predicted_test[0].numpy())
model.calc_metrics()


print("Train_MeanSquaredError: ", model.train_mse.result().numpy())
print("Train_Keras Accuracy: ", model.train_accuracy.result().numpy())
print("Train_Keras Precision: ", model.train_precision.result().numpy())
print('--------------------------------------------------------')
print("Test_MeanSquaredError: ", model.test_mse.result().numpy())
print("Test_Keras Accuracy: ", model.test_accuracy.result().numpy())
print("Test_Keras Precision: ", model.test_precision.result().numpy())

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].plot([e for e in range(model.epochs)], model.loss_history)
ax[0].set_xlabel('Loss History')
ax[0].set_ylabel('Epochs')

ax[1].scatter(X_train2[:, 0], y_train2[:])
ax[1].plot(X_train2[:, 0], model.predicted[:], color='r')
ax[1].set_ylim([-200, 200])
ax[1].set_xlim(-3,3)

ax[2].scatter(X_test2[:, 0], y_test2[:])
ax[2].plot(X_test2[:, 0], model.predicted_test[:], color='r')
ax[2].set_ylim([-200, 200])
ax[2].set_xlim(-3,3)

fig3d = plt.figure()
ax3d = fig.add_subplot(projection='3d')
ax3d.scatter(X_train2[:, 0], X_train2[:, 1], model.predicted[:])


plt.show()



(444, 1) * (1, 444) + (1,) = (444, 444)
(444, 1) * (1, 444) + (1,) = (444, 444)
(444, 444) * (444, 444) + (1,) = (444, 444)
(444, 444) * (444, 444) + (1,) = (444, 444)
(444, 444) * (444, 444) + (1,) = (444, 444)
(444, 444) * (444, 444) + (1,) = (444, 444)
(444, 444) * (444, 444) + (1,) = (444, 444)
(444, 444) * (444, 444) + (1,) = (444, 444)
